In [1]:
%%configure -f

{"conf":
     {"spark.pyspark.python":"/home/hadoop/epidemics-venv/bin/python3",
      "spark.pyspark.virtualenv.bin.path":"/home/hadoop/epidemics-venv/bin",
      "spark.pyspark.python":"/home/hadoop/epidemics-venv/bin/python3",
      "spark.dynamicAllocation.enabled":"true",
      "spark.executor.memory":"16g",
      "spark.driver.memory":"16g",
      "spark.executor.cores":"6",
      "spark.driver.cores":"6",
      "spark.driver.maxResultSize":"20g",
      "livy.server.session.timeout-check" :"false"}
}

In [8]:
import pandas as pd
import geopandas as gpd
import json
import shapely
from shapely.geometry import Point, MultiPoint, Polygon, shape, box
from shapely.ops import unary_union, transform

from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
from pyspark.sql import functions as F
from pyspark.sql.types import DoubleType, ArrayType

import daphme as dm
from daphme import cleaning

from sedona.spark import *

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
spark = SparkSession.\
    builder.\
    master("local[*]").\
    appName("Example1").\
    getOrCreate()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Load in a bounding box for Philadelphia

In [4]:
import boto3
from io import BytesIO

s3 = boto3.client('s3')
bucket_name = 'upenn-seas-wattscovid19lab'
object_key = 'paco/geometry/Census_Tracts_2010.geojson'

obj = s3.get_object(Bucket=bucket_name, Key=object_key)
geojson_data = json.load(BytesIO(obj['Body'].read()))

features = geojson_data['features']

polygons = [shape(feature["geometry"]).buffer(0) for feature in features if feature["properties"]["GEOID10"][:5] == "42101"]
phila_poly = unary_union(polygons).buffer(0.0015).simplify(0.0015)
phila_box = box(*phila_poly.bounds)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

/home/hadoop/epidemics-venv/lib64/python3.7/site-packages/boto3/compat.py:82: PythonDeprecationWarning: Boto3 will no longer support Python 3.7 starting December 13, 2023. To continue receiving service updates, bug fixes, and security updates please upgrade to Python 3.8 or later. More information can be found here: https://aws.amazon.com/blogs/developer/python-support-policy-updates-for-aws-sdks-and-tools/
  warnings.warn(warning, PythonDeprecationWarning)

## Intro to DAPHME

In this script, we will demonstrate some of DAPHME's functionalities for analyzing human mobility data. Our analysis will focus on the month of February for the city of Philadelphia. Using a dataset that originally contains [60000] users. 

In our analysis, we will perform three different kinds of operations, which are common in the analyses in these types of data. 
* **Importing**, which optionally converts the datetime to a specified timezone and project the coordinates into ESPG:3857 (which has units in meters and might be appropriate for a local analysis not too far from the equator). 
* **Coarse Filtering**, where we subset to users to make computations for tractable. In particular, we filter out users with very few days/hours with activity, and keep TODO [users with "sufficient" pings in the area]. We also filter to users with activity in a given time period.
* **Persisting**

## Load Pings

Daphme can work with a single file in a number of formats (csv, parquet, txt) but its true utility comes when working with partitions of the data, which are often organized in a directory. In this example, we specify the path of the files to be loaded.

In [5]:
gravy_df = spark.read.parquet("s3://phl-pings/gravy/")

gravy_df = gravy_df. \
    withColumn("latitude", F.col("geohashlatitude").cast(DoubleType())). \
    withColumn("longitude", F.col("geohashlongitude").cast(DoubleType()))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
#cuebiq_df = spark.read.options(header='True', inferSchema='True', delimiter=',').parquet("s3://phl-pings/cuebiq-jan-mar/")
#cuebiq_df.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
gravy_df = gravy_df.withColumn(
    "local_timestamp",
    F.from_utc_timestamp(
        F.to_timestamp(F.col("timestamp") /
                       1000),  # divide by 1000 because given in milliseconds
        F.col("timezone")  # converts timestamp from UTC to local timezone
    ))

# gravy_df = gravy_df.withColumn(
#    "timestamp",
#    F.from_utc_timestamp(
#        F.to_utc_timestamp(gravy_df.timestamp1, F.col("timezone")), # converts ts from tz to UTC
#    F.col("timezone")) # converts ts from UTC to tz

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [19]:
def clean_coords(df):
    """
    Takes in a spark dataframe with columns
        'latitude', containing latitudes in EPSG:4326
        'longitude', containing longitudes in EPSG:4326
    
    Returns a spark dataframe with a new column 'mercator_coord' containing point geometries (lat, long) in EPSG:3857
    """
    
    df.createOrReplaceTempView("df")
    df = spark.sql("""
        SELECT *,
               ST_FlipCoordinates(
                   ST_Transform(
                       ST_MakePoint(longitude, latitude), 
                       'EPSG:4326', 'EPSG:3857'
                   )
               ) AS mercator_coord
        FROM df
        """)
    return df

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
gravy_df = cleaning.clean_coords(gravy_df)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
name 'spark' is not defined
Traceback (most recent call last):
  File "/home/hadoop/epidemics-venv/lib64/python3.7/site-packages/daphme/cleaning.py", line 13, in clean_coords
    df = spark.sql("""
NameError: name 'spark' is not defined



In [ ]:
gravy_df.select("grid").distinct().count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [22]:
gravy_df.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<class 'pyspark.sql.dataframe.DataFrame'>

## Coarse Filtering

Takes in a geometry and filters to pings within the given geometry

In [20]:
testDF = gravy_df.limit(10)
testDF = testDF.select("grid", "local_timestamp", "coordinate", "mercator_coord")
testDF.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+-------------------+--------------------+--------------------+
|                grid|    local_timestamp|          coordinate|      mercator_coord|
+--------------------+-------------------+--------------------+--------------------+
|00000006-23aa-3e4...|2020-03-01 16:50:37|POINT (39.947813 ...|POINT (4858361.50...|
|00000013-ec62-3ef...|2019-10-16 05:36:06|POINT (40.013387 ...|POINT (4867887.83...|
|00000013-ec62-3ef...|2019-10-16 07:36:39|POINT (40.013387 ...|POINT (4867887.83...|
|00000013-ec62-3ef...|2019-10-16 10:36:16|POINT (40.031884 ...|POINT (4870576.65...|
|00000013-ec62-3ef...|2019-10-27 22:34:02|POINT (40.013087 ...|POINT (4867844.22...|
|00000013-ec62-3ef...|2019-10-27 23:36:07|POINT (40.013087 ...|POINT (4867844.22...|
|00000013-ec62-3ef...|2019-10-28 02:36:09|POINT (40.013087 ...|POINT (4867844.22...|
|00000013-ec62-3ef...|2019-10-28 03:36:09|POINT (40.013087 ...|POINT (4867844.22...|
|00000013-ec62-3ef...|2019-10-28 05:36:06|POINT (40.013087 ...|PO

In [32]:
# Obtain unique users with at least 1 ping in sample window
phila_box_wkt = phila_box.wkt

gravy_df.createOrReplaceTempView("gravy_df")

query = f"""
SELECT DISTINCT grid
FROM gravy_df
WHERE ST_Contains (ST_GeomFromWKT('{phila_box_wkt}'), ST_FlipCoordinates(coordinate))
"""

phl_users = spark.sql(query)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [33]:
phl_users.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

42391691